<h1><font color="#113D68" size=6>Deep Learning con Python con Python y Keras</font></h1>

<h1><font color="#113D68" size=5>Parte 4. MLP avanzado</font></h1>

<h1><font color="#113D68" size=4>1. Forecasting</font></h1>

<br><br>
<div style="text-align: right">
<font color="#113D68" size=3>Manuel Castillo Cara</font><br>

</div>

---

<a id="indice"></a>
<h2><font color="#004D7F" size=5>Índice</font></h2>

* [0. Contexto](#section0)
* [1. Introducción](#section1)
    * [1.1. Formato HDF5](#section1.1)
* [2. Guardar el modelo en JSON](#section2)
* [3. Guardar el modelo en YAML](#section3)
* [4. Guardar topología y pesos](#section4)
    * [4.1. Guardar el modelo](#section4.1)
    * [4.2. Cargar el modelo](#section4.2)

---
<a id="section0"></a>
# <font color="#004D7F" size=6> 0. Contexto</font>

En esta lección, descubrirá cómo puede guardar nuestros modelos en un archivo y volver a cargarlos para hacer predicciones. Después de completar esta lección, sabrá:
* Cómo guardar y cargar los pesos del modelo en archivos formateados HDF5.
* Cómo guardar y cargar la estructura del modelo en archivos JSON.
* Cómo guardar y cargar la estructura del modelo en archivos YAML.

In [1]:
import tensorflow as tf
# Eliminar warning
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section1"></a>
# <font color="#004D7F" size=6>1. Introducción</font>

Keras puede guardar la arquitectura de su modelo y guardar los pesos y nuestro modelo. 
1. Los pesos de los modelos se guardan en formato HDF5. 
2. El modelo se puede guardar (y cargar) utilizando dos formatos diferentes: JSON y YAML.

<a id="section1.1"></a>
# <font color="#004D7F" size=5>1.1. Formato HDF5</font>

El formato de datos jerárquico (HDF5) es un formato de almacenamiento de datos flexible y es conveniente para almacenar grandes conjuntos de valores reales, como tenemos en los pesos de las redes neuronales. Puede instalar este formato con `pip`:

In [2]:
#!pip install h5py

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section2"></a>
# <font color="#004D7F" size=6>2. Guardar el modelo en JSON</font>

Keras brinda la capacidad de describir cualquier modelo usando formato JSON con una función `to_json()`. Posteriormente, podemos cargarlo a través de `model_from_json()`.

Los pesos se guardan directamente usando la función `save_weights()` y luego se cargan usando la función `load_weights()`. 

El siguiente ejemplo entrena y evalúa un modelo. Luego, la estructura del modelo se convierte a formato JSON con `model.json` en el directorio local. Los pesos de la red se escriben en `model.h5`.

In [11]:
# MLP for Pima Indians Dataset Serialize to JSON and HDF5
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json


# load pima indians dataset
from google.colab import drive
drive.mount('/content/drive')
path= "/content/drive/MyDrive/CursoDeepLearning/Datasets/pima-indians-diabetes.csv"

#dataframe= pd.read_csv(path,header=None)
#dataset= dataframe.values

dataset= np.loadtxt(path, delimiter=',')

# split into input (X) and output (Y) variables
x= dataset[:,0:8]
y= dataset[:,8]
y

# create model
model= Sequential()
model.add(Dense(12,input_dim=8,activation='relu'))
model.add(Dense(8,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy']) 

# Fit the model
model.fit(x,y,epochs=100, batch_size=10, verbose=0)

# evaluate the model
scores= model.evaluate(x,y, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1],scores[1]*100))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
accuracy: 75.00%


In [15]:
# serialize model to JSON
model_json= model.to_json()
with open('model.json','w') as json_file:
  json_file.write(model_json)

# serialize weights to HDF5
model.save_weights('model.h5')


In [21]:
# load json and create model
json_file= open('model.json','r')
loaded_model_json= json_file.read()
json_file.close()

loaded_model= model_from_json(loaded_model_json)
loaded_model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

# load weights into new model
loaded_model.load_weights('model.h5')

# evaluate loaded model on test data
scores= loaded_model.evaluate(x,y, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1],scores[1]*100))

accuracy: 75.00%


El formato JSON del modelo tiene el siguiente aspecto:

In [22]:
cat model.json | python -mjson.tool

{
    "class_name": "Sequential",
    "config": {
        "name": "sequential_3",
        "layers": [
            {
                "class_name": "InputLayer",
                "config": {
                    "batch_input_shape": [
                        null,
                        8
                    ],
                    "dtype": "float32",
                    "sparse": false,
                    "ragged": false,
                    "name": "dense_9_input"
                }
            },
            {
                "class_name": "Dense",
                "config": {
                    "name": "dense_9",
                    "trainable": true,
                    "batch_input_shape": [
                        null,
                        8
                    ],
                    "dtype": "float32",
                    "units": 12,
                    "activation": "relu",
                    "use_bias": true,
                    "kernel_initializer": {
                     

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section3"></a>
# <font color="#004D7F" size=6>3. Guardar el modelo en YAML</font>

El modelo se guarda como YAML en el archivo `model.yaml` y luego se carga en un nuevo modelo a través de la función `model_from_yaml()`. 

Los pesos se manejan de la misma manera en formato HDF5, `model.h5`.

In [24]:
# MLP for Pima Indians Dataset Serialize to YAML and HDF5
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_yaml

# load pima indians dataset
from google.colab import drive
drive.mount('/content/drive')
path= "/content/drive/MyDrive/CursoDeepLearning/Datasets/pima-indians-diabetes.csv"

dataset= np.loadtxt(path, delimiter=',')

# split into input (X) and output (Y) variables
x= dataset[:,0:8]
y= dataset[:,8]
y

# create model
model= Sequential()
model.add(Dense(12,input_dim=8,activation='relu'))
model.add(Dense(8,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])

# Fit the model
model.fit(x,y,epochs=100, batch_size=10, verbose=0)

# evaluate the model
scores= model.evaluate(x,y, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1],scores[1]*100))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
accuracy: 71.48%


In [25]:
# serialize model to YML  #### no soportado en Tensorflow 2.6 usar json
model_yaml= model.to_yaml()
with open('model.yaml','w') as yaml_file:
  yaml_file.write(model_yaml)

# serialize weights to HDF5
model.save_weights('model.h5')

RuntimeError: ignored

In [ ]:
# load YAML and create model
yaml_file= open('model.yaml','r')
loaded_model_yaml= yaml_file.read()
yaml_file.close()

loaded_model= model_from_yaml(loaded_model_yaml)
loaded_model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

# load weights into new model
loaded_model.load_weights('model.h5')

# evaluate loaded model on test data
scores= loaded_model.evaluate(x,y, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1],scores[1]*100))


Modelo cargado
accuracy: 77.60%


El modelo descrito en formato YAML tiene el siguiente aspecto:

In [ ]:
print(loaded_model_yaml)

backend: tensorflow
class_name: Sequential
config:
  layers:
  - class_name: InputLayer
    config:
      batch_input_shape: !!python/tuple
      - null
      - 8
      dtype: float32
      name: dense_6_input
      ragged: false
      sparse: false
  - class_name: Dense
    config:
      activation: relu
      activity_regularizer: null
      batch_input_shape: !!python/tuple
      - null
      - 8
      bias_constraint: null
      bias_initializer:
        class_name: Zeros
        config: {}
      bias_regularizer: null
      dtype: float32
      kernel_constraint: null
      kernel_initializer:
        class_name: GlorotUniform
        config:
          seed: null
      kernel_regularizer: null
      name: dense_6
      trainable: true
      units: 12
      use_bias: true
  - class_name: Dense
    config:
      activation: relu
      activity_regularizer: null
      bias_constraint: null
      bias_initializer:
        class_name: Zeros
        config: {}
      bias_regularizer: nu

<a id="section4"></a>
# <font color="#004D7F" size=6>4. Guardar topología y pesos</font>

Guardar el modelo incluye todo sobre el modelo, que incluye:
* Pesos.
* Arquitectura.
* Detalles de compilación del modelo (pérdidas y métricas).
* Estado del optimizador de modelo.

Esto significa que podemos cargar y usar el modelo directamente.

<a id="section4.1"></a>
# <font color="#004D7F" size=5>4.1. Guardar el modelo</font>

La función `save()` guarda el modelo especificando el nombre del archivo. 

El ejemplo muestra como ajustar un modelo, evaluarlo y guardarlo en el archivo `model.h5`.

In [28]:
# MLP for Pima Indians Dataset saved to single file
import numpy as np
from keras.models import Sequential
from keras.layers import Dense


# load pima indians dataset
from google.colab import drive
drive.mount('/content/drive')
path= "/content/drive/MyDrive/CursoDeepLearning/Datasets/pima-indians-diabetes.csv"

dataset= np.loadtxt(path, delimiter=',')

# split into input (X) and output (Y) variables
x= dataset[:,0:8]
y= dataset[:,8]
y

# create model
model= Sequential()
model.add(Dense(12,input_dim=8,activation='relu'))
model.add(Dense(8,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])

# Fit the model
model.fit(x,y,epochs=100, batch_size=10, verbose=0)

# evaluate the model
scores= model.evaluate(x,y, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1],scores[1]*100))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
accuracy: 75.39%


In [30]:
# save model and architecture to single file
model.save('model.h5')
print('Modelo guardado')

Modelo guardado


<a id="section4.2"></a>
# <font color="#004D7F" size=5>4.2. Cargar el modelo</font>

Su modelo guardado se puede cargar más tarde llamando a la función `load_model()` y pasando el nombre del archivo. La función devuelve el modelo con la misma arquitectura y pesos. En este caso, cargamos el modelo, resumimos la arquitectura y la evaluamos en el mismo conjunto de datos para confirmar que los pesos y la arquitectura son los mismos.

In [37]:
# load and evaluate a saved model
from keras.models import load_model

# load model
loaded_model= load_model('model.h5')

# summarize model.
loaded_model.summary()


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 12)                108       
_________________________________________________________________
dense_19 (Dense)             (None, 8)                 104       
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 9         
Total params: 221
Trainable params: 221
Non-trainable params: 0
_________________________________________________________________


In [39]:
# load dataset
from google.colab import drive
drive.mount('/content/drive')
path= "/content/drive/MyDrive/CursoDeepLearning/Datasets/pima-indians-diabetes.csv"

dataset= np.loadtxt(path, delimiter=',')

# split into input (X) and output (Y) variables
x= dataset[:,0:8]
y= dataset[:,8]

# evaluate the model
scores= loaded_model.evaluate(x,y, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1],scores[1]*100))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
accuracy: 75.39%


<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-coffee" aria-hidden="true" style="color:#004D7F"></i> </font></div>